In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
datos = np.loadtxt('datos_observacionales.dat')
t = datos[:,0]
x = datos[:,1]
y = datos[:,2]
z = datos[:,3]

x_obs = x

In [17]:
def model(x,param):
    n_param = 3
    y = 0
    for i in range(n_param):
        y += param[i] * x**i
    return y 
    
def loglikelihood(x_obs, y_obs, sigma_y_obs, param):
    d = y_obs -  model(x_obs, param)
    d = d/sigma_y_obs
    d = -0.5 * np.sum(d**2)
    return d

def logprior(param): 
    p = -np.inf
    if( 0< np.absolute(param) < 30 ): 
        p = 0.0
    return p

In [18]:
def divergence_loglikelihood(x_obs, y_obs, sigma_y_obs, param):
    n_param = 3
    div = np.ones(n_param)
    delta = 1E-5
    for i in range(n_param):
        delta_parameter = np.zeros(n_param)
        delta_parameter[i] = delta
        
        div[i] = loglikelihood(x_obs, y_obs, sigma_y_obs, param + delta_parameter) 
        div[i] = div[i] - loglikelihood(x_obs, y_obs, sigma_y_obs, param - delta_parameter)
        div[i] = div[i]/(2.0 * delta)
    return div

In [19]:
N = 50000

lista_sigma = [np.random.random()]
lista_rho = [np.random.random()]
lista_beta = [np.random.random()]

logposterior = [loglikelihood(x_obs, lista_sigma[0], lista_rho[0], lista_beta[0]) + logprior(lista_sigma[0], lista_rho[0], lista_beta[0])]
sigma_delta = 1.0

for i in range(1,N):

    propuesta_sigma  = lista_v[i-1] + np.random.normal(loc=0.0, scale=sigma_delta)
    propuesta_rho = lista_rho[i-1] + np.random.normal(loc=0.0, scale=sigma_delta)
    propuesta_beta = lista_rho[i-1] + np.random.normal(loc=0.0, scale=sigma_delta)
    
    logposterior_viejo = loglikelihood(x_obs, lista_sigma[i-1], lista_rho[i-1], lista_beta[i-1]) + logprior(lista_sigma[i-1], lista_rho[i-1], lista_beta[i-1])
    logposterior_nuevo = loglikelihood(x_obs, propuesta_sigma, propuesta_rho, propuesta_beta) + logprior(propuesta_sigma, propuesta_rho, propuesta_beta)

    r = min(1,np.exp(logposterior_nuevo-logposterior_viejo))

    alpha = np.random.random()
    
    if(alpha < r):
            lista_sigma.append(propuesta_sigma)
            lista_rho.append(propuesta_rho)
            lista_beta.append(propuesta_beta)
            logposterior.append(logposterior_nuevo)
    else:
        lista_sigma.append(lista_sigma[i-1])
        lista_rho.append(lista_rho[i-1])
        lista_beta.append(lista_beta[i-1])
        logposterior.append(logposterior_viejo)


lista_sigma = np.array(lista_sigma)
lista_rho = np.array(lista_rho)
lista_beta = np.array(lista_beta)
logposterior = np.array(logposterior)

TypeError: 'float' object is not subscriptable